1. 더미 데이터 불러오기

In [ ]:
import pandas as pd

# 예시 CSV (idea_id,title,body)
df = pd.read_csv("./data/ideas_sample_1000.csv").fillna("")
df["text"] = df["title"].str.strip() + " " + df["body"].str.strip()
print(df.head(2))

    idea_id        title                                            body  좋아요  \
0  idea_001   로봇 바리스타 카페  무인 로봇 팔이 주문과 커피 제조를 담당해 24시간 운영하는 스마트 카페 아이디어.   39   
1  idea_002  반려동물 1인 미용실      셀프 그루밍 부스와 전문가 예약 서비스를 결합한 동네 소형 반려동물 미용실.   16   

   싫어요                                               text  
0    0  로봇 바리스타 카페 무인 로봇 팔이 주문과 커피 제조를 담당해 24시간 운영하는 스...  
1   15  반려동물 1인 미용실 셀프 그루밍 부스와 전문가 예약 서비스를 결합한 동네 소형 반...  


2. 전처리(클렌징)

In [3]:
import re

def clean(txt: str) -> str:
    txt = re.sub(r"http\S+|www\S+", " ", txt)            # URL
    txt = re.sub(r"[^\w가-힣\s]", " ", txt)              # 특수문자
    txt = re.sub(r"\s+", " ", txt).strip()               # 중복 공백
    return txt.lower()

df["clean"] = df["text"].apply(clean)

3. SimCSE 임베딩

In [4]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

MODEL_ID = "BM-K/KoSimCSE-roberta"      # unsupervised 버전
embedder = SentenceTransformer(MODEL_ID)

BATCH = 256
emb_list = []
for i in tqdm(range(0, len(df), BATCH)):
    batch = df["clean"].iloc[i : i + BATCH].tolist()
    embs  = embedder.encode(batch, batch_size=len(batch),
                            normalize_embeddings=True)
    emb_list.extend(embs)
import numpy as np
emb = np.vstack(emb_list).astype("float32")              # (N, 768)

No sentence-transformers model found with name BM-K/KoSimCSE-roberta. Creating a new one with mean pooling.
100%|██████████| 1/1 [00:03<00:00,  3.41s/it]


normalize_embeddings=True → 이미 L2 노멀라이즈된 벡터라 Inner Product = 코사인.

4. FAISS 인덱스 생성

In [5]:
import faiss

d = emb.shape[1]                     # 768
index = faiss.IndexFlatIP(d)         # 작은 데이터셋은 Flat IP로 충분
index.add(emb)                       # 전체 아이디어 삽입

5. 유사 아이디어 검색 함수

In [6]:
def find_similar(query: str, top_k: int = 5):
    q_emb = embedder.encode([clean(query)],
                            normalize_embeddings=True).astype("float32")
    D, I = index.search(q_emb, top_k)      # D: 코사인, I: 행 인덱스
    return list(zip(I[0], D[0]))           # [(idx, score), …]

print(find_similar("로봇 바리스타 카페 창업 아이디어"))

[(0, 0.68017864), (7, 0.46029842), (8, 0.41422057), (1, 0.40591317), (9, 0.40527844)]


6. HDBSCAN 클러스터링

In [11]:
import hdbscan, joblib, numpy as np

n = len(df)                                     # 현재 데이터 크기
min_cluster = max(2, int(0.2 * n))              # 20% 또는 최소 2
min_samples = min(min_cluster, n - 1)

clusterer = hdbscan.HDBSCAN(
        metric="euclidean",
        min_cluster_size=min_cluster,
        min_samples=min_samples,
        prediction_data=True
).fit(emb)

df["cluster"] = clusterer.labels_
print(df["cluster"].value_counts())
joblib.dump(clusterer, "hdbscan.pkl")

cluster
 1    5
-1    3
 0    2
Name: count, dtype: int64


/Users/leeyoungho/miniforge3/envs/ai_3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/leeyoungho/miniforge3/envs/ai_3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


['hdbscan.pkl']

-----

여기부터는 .py 에서 진행

7. 새 글 추가 → 실시간 검색 + 인덱스 갱신

In [12]:
def add_idea(new_row: dict, search_k: int = 5):
    """
    새 아이디어 1건을
      1) 전처리·임베딩
      2) FAISS 검색 → 유사도 top-k 반환
      3) 인덱스·데이터프레임·임베딩 배열 업데이트
    """
    global emb, df, index

    # 1) 전처리 + 임베딩
    cleaned = clean(new_row["title"] + " " + new_row["body"])
    vec     = embedder.encode([cleaned], normalize_embeddings=True).astype("float32")

    # 2) 유사도 검색
    D, I = index.search(vec, search_k)
    similar = [(int(idx), float(score)) for idx, score in zip(I[0], D[0])]

    # 3-A) 인덱스·임베딩 배열 업데이트
    index.add(vec)                          # FAISS에 즉시 반영
    emb = np.vstack([emb, vec])             # ndarray 확장

    # 3-B) 데이터프레임 업데이트
    #     ★ 여기서 오류가 났던 부분 → 중괄호 개수 수정
    df = pd.concat(
        [df, pd.DataFrame([ new_row | {"clean": cleaned} ])],
        ignore_index=True
    )

    return similar

Note: HDBSCAN은 증분 학습이 불가하므로
cron or Airflow로 5분마다 전체 재빌드 (clusterer.fit(emb))를 돌리면 됩니다.

8. 프론트 연동 초간단 API

In [14]:
import sys
!{str(sys.executable)} -m pip install fastapi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached starlette-0.46.2-py3-none-any.whl.metadata (6.2 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
Using cached pydantic-2.11.7-py3-none-any.whl (444 kB)
Using cached pydantic_core-2.33.2-cp312-cp312-macosx_11_0_arm64.whl (1.8 MB)
Using cached starlette-0.46.2-py3-none-any.whl (72 kB)
Using cached anyio-4.9.0-py3-none-any.whl (100 kB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Using cached typing_inspection-0.4.1-py3-none-any.whl (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [fastapi]m5/8 [anyio]ic]


In [15]:
from fastapi import FastAPI
app = FastAPI()

@app.post("/submit")
def submit(idea: dict):
    sim = add_idea(idea)
    # 코사인 0.7↑ + 같은 cluster 아이템만 추천
    recs = [idx for idx, sc in sim if sc > 0.7 and
            df.loc[idx, "cluster"] == df.iloc[-1]["cluster"]]
    return {"similar_ids": recs}

In [16]:
import sys
!{str(sys.executable)} -m pip install fastapi uvicorn pydantic[dotenv] requests

zsh:1: no matches found: pydantic[dotenv]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
